In [1]:
import os
import sys
import yaml
import json
import numpy as np

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
#Get data path from config 
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['contants']['mb_div']

#Get dyype for meta file 
meta_data_file = os.path.join(data_path, cfg['data']['metadata'])
with open(meta_data_file) as f:
    meta_data = json.load(f)

In [2]:
import pandas as pd
print(pd.__version__)
import datetime as dt

0.25.1


## Tiny Data
150 rows, 3 cols

In [35]:
%%time

iris_file = os.path.join(data_path, cfg['data']['iris_csv'])
iris = pd.read_csv(iris_file)

CPU times: user 3.95 ms, sys: 2.72 ms, total: 6.67 ms
Wall time: 4.89 ms


In [36]:
print("data frame memory(MB):", round(sys.getsizeof(iris))/mb_div)
print("data frame memory(MB):", round(iris.memory_usage(deep = "True").sum()/mb_div,5))
print(iris.shape)
del(iris)

data frame memory(MB): 0.01476
data frame memory(MB): 0.01473
(150, 5)


 ## Bigger Data
55 million rows, 8 cols

In [ ]:
%%time

nyc_taxi_fare_file = os.path.join(data_path, cfg['data']['kaggle_nyc'])
nyc_taxi_fare_dtype = dict(zip(meta_data['kaggle_nyc']['colnames'], meta_data['kaggle_nyc']['dtype']))
nyc_taxi_fare = pd.read_csv(nyc_taxi_fare_file, 
                            dtype = nyc_taxi_fare_dtype,
                             parse_dates = meta_data['kaggle_nyc']['datecols'])

In [ ]:
print("data frame memory(MB):", round(sys.getsizeof(nyc_taxi_fare)/mb_div,3))
print(nyc_taxi_fare.shape)
nyc_taxi_fare.head()

In [ ]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].agg(count = len, 
                                                              mean = np.mean,
                                                              std = np.std,
                                                              min = np.min,
                                                              median = np.median,
                                                              max = np.max)

#How to get quantile ? 

In [ ]:
%%time
nyc_taxi_fare.groupby(['passenger_count'])['fare_amount'].describe()

## PANDAS INTERESTING BITS ##

In [32]:
del(nyc_taxi_fare)

In [119]:
nyc_taxi_fare.groupby(nyc_taxi_fare['pickup_datetime'].dt.year).agg('count')

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
pickup_datetime,,,,,,,,
2009,1531,1531,1531,1531,1531,1531,1531,1531
2010,1538,1538,1538,1538,1538,1538,1538,1538
2011,1560,1560,1560,1560,1560,1560,1560,1560
2012,1640,1640,1640,1640,1640,1640,1640,1640
2013,1585,1585,1585,1585,1585,1585,1585,1585
2014,1475,1475,1475,1475,1475,1475,1475,1475
2015,671,671,671,671,671,671,671,671


## Too Big Data

46 million rows, 32 columns 
Need tp come up with a better example


In [73]:
%%time

dc_taxi_file_quad = os.path.join(data_path, cfg['data']['dc_taxi_2017_quad'])
dc_taxi_quad = pd.read_csv(dc_taxi_file_quad)

CPU times: user 6min 45s, sys: 6min 44s, total: 13min 30s
Wall time: 21min 40s


In [77]:
print("pandas table memory(MB)", round(sys.getsizeof(dc_taxi_quad)/mb_div,3))
print(dc_taxi_quad.shape)

NameError: name 'sys' is not defined

In [75]:
del(dc_taxi_quad)